In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [11]:
training_data = datasets.FashionMNIST(root="data", train=True,  download=True, transform=ToTensor())
test_data     = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader  = DataLoader(test_data,     batch_size=64)

In [3]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNet()
print(model)

NeuralNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [4]:
# hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5

Common loss functions include `nn.MSELoss` (Mean Square Error) for regression tasks, and `nn.NLLLoss` (Negative Log Likelihood) for classification. `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`

In [5]:
loss_fn = nn.CrossEntropyLoss()

Stochastic Gradient Descent (SGD), ADAM, and RMSProp are different optimizations:

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # reset the gradients of model parameters. 
        # Gradients by default add up; to prevent double-counting, 
        # we explicitly zero them at each iteration.
        optimizer.zero_grad()
        # Backpropagate the prediction loss with a call to loss.backward(). 
        # PyTorch deposits the gradients of the loss w.r.t. each parameter.
        loss.backward()
        # adjust the parameters by the gradients collected in the backward pass
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [9]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error:\n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301534 [    0/60000]
loss: 2.289587 [ 6400/60000]
loss: 2.265394 [12800/60000]
loss: 2.261472 [19200/60000]
loss: 2.254048 [25600/60000]
loss: 2.221114 [32000/60000]
loss: 2.230749 [38400/60000]
loss: 2.196828 [44800/60000]
loss: 2.200105 [51200/60000]
loss: 2.158045 [57600/60000]
Test Error:
 Accuracy: 50.2%, Avg loss: 2.155983

Epoch 2
-------------------------------
loss: 2.169783 [    0/60000]
loss: 2.157080 [ 6400/60000]
loss: 2.097878 [12800/60000]
loss: 2.112527 [19200/60000]
loss: 2.071290 [25600/60000]
loss: 2.009252 [32000/60000]
loss: 2.036123 [38400/60000]
loss: 1.960486 [44800/60000]
loss: 1.968283 [51200/60000]
loss: 1.886993 [57600/60000]
Test Error:
 Accuracy: 58.6%, Avg loss: 1.886966

Epoch 3
-------------------------------
loss: 1.928085 [    0/60000]
loss: 1.892460 [ 6400/60000]
loss: 1.771699 [12800/60000]
loss: 1.808454 [19200/60000]
loss: 1.711367 [25600/60000]
loss: 1.656240 [32000/60000]
loss: 1.677491 [38400/6000